# RNN - long sentence

## 1) 필요한 모듈을 import 합니다

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

## 2) maunal seed를 고정시킵니다

In [4]:
torch.manual_seed(1)

## 3) 학습 문장을 입력합니다

In [5]:
sent = ("This story is a very contemporary one about the gap between rich and poor. And essentially, "
        "it's about capitalism. And I think it's undeniable that the heart of capitalism is the U.S. "
        "And so, I think it's been very gratifying to see the strong reaction here.")

In [6]:
sent

"This story is a very contemporary one about the gap between rich and poor. And essentially, it's about capitalism. And I think it's undeniable that the heart of capitalism is the U.S. And so, I think it's been very gratifying to see the strong reaction here."

## 4) 문자 셋과 딕셔너리를 생성합니다

In [7]:
set_char = list(set(sent))
dic_char = {c : i for i, c in enumerate(set_char)}

In [8]:
print(dic_char)

{'p': 0, 'h': 1, 't': 2, ' ': 3, 'f': 4, 'm': 5, 'v': 6, 'u': 7, 'i': 8, 'y': 9, 's': 10, 'U': 11, 'T': 12, ',': 13, 'A': 14, "'": 15, 'S': 16, 'I': 17, '.': 18, 'd': 19, 'n': 20, 'g': 21, 'k': 22, 'c': 23, 'a': 24, 'r': 25, 'b': 26, 'o': 27, 'w': 28, 'e': 29, 'l': 30}


## 5) hyper parameter를 지정합니다

In [9]:
dic_size = len(dic_char)
hidden_size = len(dic_char)
seq_length = 10
learning_rate = 1e-3
epochs= 1000

## 6) data setting

In [10]:
x_data = []
y_data = []

for i in range(0, len(sent) - seq_length):
    x_str = sent[i:i+seq_length]
    y_str = sent[i+1:i+seq_length+1]
    print(i, x_str, '->', y_str)
    
    x_data.append([dic_char[c] for c in x_str])
    y_data.append([dic_char[c] for c in y_str])

0 This story -> his story 
1 his story  -> is story i
2 is story i -> s story is
3 s story is ->  story is 
4  story is  -> story is a
5 story is a -> tory is a 
6 tory is a  -> ory is a v
7 ory is a v -> ry is a ve
8 ry is a ve -> y is a ver
9 y is a ver ->  is a very
10  is a very -> is a very 
11 is a very  -> s a very c
12 s a very c ->  a very co
13  a very co -> a very con
14 a very con ->  very cont
15  very cont -> very conte
16 very conte -> ery contem
17 ery contem -> ry contemp
18 ry contemp -> y contempo
19 y contempo ->  contempor
20  contempor -> contempora
21 contempora -> ontemporar
22 ontemporar -> ntemporary
23 ntemporary -> temporary 
24 temporary  -> emporary o
25 emporary o -> mporary on
26 mporary on -> porary one
27 porary one -> orary one 
28 orary one  -> rary one a
29 rary one a -> ary one ab
30 ary one ab -> ry one abo
31 ry one abo -> y one abou
32 y one abou ->  one about
33  one about -> one about 
34 one about  -> ne about t
35 ne about t -> e about th
36

In [11]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

## 7) model architecture를 설계합니다

In [12]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim, bias=True)
        
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

## 8) model, criterion, optimizer를 선언합니다

In [13]:
net = Net(dic_size, hidden_size, 2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

## 9) training을 시작합니다

In [56]:
epochs = 1000
for epoch in range(1, epochs+1):
    output = net(X)
    
    loss = criterion(output.view(-1, dic_size), Y.view(-1))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        results = output.argmax(dim=2)
        predict_str = ''

        for j, result in enumerate(results):

            if j == 0:
                predict_str += ''.join([set_char[t] for t in result])
            else:
                predict_str += set_char[result[-1]]

        print(epoch, '번째 출력 (loss:', loss.item(),'\n', predict_str, 
              '\n-----------------------------------------------------------------------------------------------')

100 번째 출력 (loss: 2.6374568939208984 
  e   aee   ae  te   iae          ie    e   tae   t tet      ee   e   ee    at tet    t      ee   ee   tat t t t   t t t tee   iee   et      e   e   ae   a   te  ae t   t   e   e   t   at t t   tee   iee   ee   ee  iae  t     tee    t e   ae   ae   ee   ee   
-----------------------------------------------------------------------------------------------
200 번째 출력 (loss: 1.624925136566162 
 ees t ory it t deny iorteateraty on  aboot the g pitetieen ierh ind toont And trtenthalyyy it's about thpithlisy  And t thenk it's aed adeple then the  ea t on inpithlisy is the t d  And too i thenk it's aee  iery iratityind tortea the  eae k ae ition har   
-----------------------------------------------------------------------------------------------
300 번째 출력 (loss: 0.9804033637046814 
 eis seony is a veny corte porary ont about the gap oetheen rirh and poor. And totentially  it's about tapitalism  And t thenk it's andeniable that the heart of capitalism is the

# Full code

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

torch.manual_seed(1)

sent = ("Chancellor Angela Merkel's party will vote on whether all schools in Germany "
        'should fly the national flag outside their buildings, alongside the flag of the appropriate state' 
        'and that of the European Union, following an initiative launched by one regional organization '
        'of the Christian Democratic Union (CDU)')

set_char = list(set(sent))
dic_char = {c:i for i, c in enumerate(set_char)}

dic_size = len(dic_char)
hidden_size = len(dic_char)
seq_length = 10
learning_rate = 1e-3
epochs = 2000

x_data = []
y_data = []

for i in range(0, len(sent)-seq_length):
    x_str = sent[i:i+seq_length]
    y_str = sent[i+1:i+seq_length+1]
    
    x_data.append([dic_char[c] for c in x_str])
    y_data.append([dic_char[c] for c in y_str])
    
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
    
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim, bias=True)
        
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x
    
net = Net(dic_size, hidden_size, 2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

for epoch in range(1, epochs+1):
    output = net(X)
    
    loss = criterion(output.view(-1, dic_size), Y.view(-1))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 200 == 0:
        results = output.argmax(dim=2)
        predict_str = ''
        
        for j, result in enumerate(results):
            
            if j == 0:
                predict_str += ''.join([set_char[t] for t in result])
                
            else:
                predict_str += set_char[result[-1]]
                
        print('{:4d}번째 출력 (loss: {:.6f}):\n {} {}'.format(epoch, loss.item(), predict_str,
             '\n-------------------------------------------------------------------------------'))

 200번째 출력 (loss: 1.928903):
 ee  hanan snioln th tel s fea i siot oene on the he  on  snhenl  sn th  an  shenal ol  she ohtion l oll  on iioe the n oniln ng  ionen s ne the oart on the onraenrioti sihaean  shetion the ohronean snion  shn lting on on tiotion oet  hel sn one oe ion n on  n  ation on the the o ion oe oneatio onion the s 
-------------------------------------------------------------------------------
 400번째 출력 (loss: 0.758164):
 hrnchlaor Angela ferkel's party will vote on whether all schools in Uermany should fly the national flag oftside their buildings, alongside the flag of the oppropriate staleand that of the ouropean Union  following an initiatice launched by one regional organization of the shristian Democratic Union wCDUw 
-------------------------------------------------------------------------------
 600번째 출력 (loss: 0.459276):
 hrncellor Angela Merkel's party will vote on whether all schools in Germany should fly the national flag outside their buildings, alongsi